# Projet Final

In [2]:
import requests
import datetime
import pandas as pd
from bs4 import BeautifulSoup
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import unicodedata

### Aquisition des données


##### Acquisition des classements journaliers. 

In [ ]:

url_template = "https://www.vendeeglobe.org/download-race-data/vendeeglobe_{}_{}.xlsx"

begin_datetime = datetime.datetime(year=2020, month=11, day=8, hour=14)
end_datetime = datetime.datetime(year=2021, month=3, day=5, hour=8)
time_delta = datetime.timedelta(hours=1)

files_count = 0
while begin_datetime <= end_datetime:
    try:
        url = url_template.format(begin_datetime.date().strftime("%Y%m%d"), begin_datetime.time().strftime("%H%M%S"))
        r = requests.get(url)
        if r.status_code == requests.codes.ok:
            files_count +=1
            with open("./datasets/"+url.split('/')[-1], 'wb') as f:
                f.write(r.content)
    except:
        pass
    begin_datetime += time_delta

print("{} files downloaded".format(files_count))


##### Aquisition des spécifications des voiliers

In [3]:
def remove_unit(s, u):
    return s.replace(u,'').strip()

r = requests.get("https://www.vendeeglobe.org/fr/glossaire")
soup = BeautifulSoup(r.content, "html.parser")
soup.prettify()
specs = []
for s in soup.find_all("div", {"class": "boats-list__popup-infos"}):
    d = {}
    d["Skipper"] = s.find_previous("span").text
    d["Bateau"] = s.find("h3", {"class": "boats-list__popup-title"}).text
    for l in s.find_all("li"):
        text = l.text.split(':')
        d[text[0].strip()] = text[1].strip()
    specs.append(d)
df_spec = pd.DataFrame(specs)
#df_spec["Date de lancement"] = pd.to_datetime(df_spec["Date de lancement"] , format="%d %B %Y")
df_spec["Longueur"] = df_spec["Longueur"].apply(lambda x: x.split('m')[0].replace(',', '.').strip()).astype(float)
df_spec["Largeur"] = df_spec["Largeur"].apply(lambda x: x.split('m')[0].replace(',', '.').strip()).astype(float)
df_spec["Tirant d'eau"] = df_spec["Tirant d'eau"].apply(lambda x: x.split('m')[0].replace(',', '.').strip()).astype(float)
df_spec["Déplacement (poids)"] = pd.to_numeric(df_spec["Déplacement (poids)"].apply(lambda x: x.split('t')[0].replace(',', '.').strip()), errors="coerce")
df_spec["Hauteur mât"] = pd.to_numeric(df_spec["Hauteur mât"].apply(lambda x: x.replace('m','').replace(',', '.').strip()), errors="coerce")
df_spec["Surface de voiles au près"] = pd.to_numeric(df_spec["Surface de voiles au près"].apply(lambda x: x.split('m')[0].replace(',', '.').strip()), errors="coerce")
df_spec["Surface de voiles au portant"] = pd.to_numeric(df_spec["Surface de voiles au portant"].apply(lambda x: x.split('m')[0].replace(',', '.').strip()), errors="coerce")
df_spec = df_spec.rename(columns={"Longueur": "Longeur (m)", "Largeur": "Largeur (m)", "Tirant d'eau": "Tirant d'eau (m)", "Déplacement (poids)": "Déplacement (poids en tonne)",\
     "Hauteur mât":"Hauteur mât (m)", "Surface de voiles au près": "Surface de voiles au près (m2)", "Surface de voiles au portant": "Surface de voiles au portant (m2)"})

df_spec.head()

,Skipper,Bateau,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longeur (m),Largeur (m),Tirant d'eau (m),Déplacement (poids en tonne),Nombre de dérives,Hauteur mât (m),Voile quille,Surface de voiles au près (m2),Surface de voiles au portant (m2)
0,Fabrice AMEDEO,NEWREST - ART & FENÊTRES,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,01 Août 2015,18.28,5.85,4.5,7.0,foils,29.0,monotype,320,570
1,Romain ATTANASIO,PURE - Best Western®,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),08 Mars 2007,18.28,5.80,4.5,9.0,2,28.0,acier forgé,280,560
2,Alexia BARRIER,TSE - 4MYPLANET,FRA72,"Famille Mary-Etamine du Lys, Initiatives Coeur...",Marc Lombard,MAG France,01 Mars 1998,18.28,5.54,4.5,9.0,2,29.0,acier,260,580
3,Yannick BESTAVEN,Maître CoQ IV,17,Safran 2 - Des Voiles et Vous,Verdier - VPLP,CDK Technologies,12 Mars 2015,18.28,5.80,4.5,8.0,foils,29.0,acier mécano soudé,310,550
4,Jérémie BEYOU,CHARAL,08,NaN,VPLP,CDK Technologies,18 Août 2018,18.28,5.85,4.5,8.0,foils,29.0,acier,320,600


##### N.B: Nous traitons uniquement les données du 8 novembre 2020 au 27 janvier 2021

In [5]:
df_ranks = pd.DataFrame()
count = 0
for f_name in os.listdir("datasets/"):
    try:
        df = pd.read_excel("datasets/"+f_name, header=None, usecols="B:U", skiprows=lambda x: x>37)
        df.iloc[3,[3,4,5]] = df.iloc[4,[3,4,5]]
        df.iloc[4,5] = df.iloc[4,3] =  df.iloc[4,4] = ''
        df.iloc[3:5] = df.iloc[3:5].fillna('')
        df.iloc[3, [7,8,9]] = df.iloc[3,6]
        df.iloc[3, [11,12,13]] = df.iloc[3,10]
        df.iloc[3, [15,16,17]] = df.iloc[3,14]
        df.iloc[3] = df.iloc[3].apply(lambda x: x.split('\n')[0])
        df.iloc[4] = df.iloc[4].apply(lambda x: x.split('\n')[0])
        df.columns= pd.MultiIndex.from_tuples(zip(*df.iloc[3:5].to_records(index=False).tolist()))
        df = df.iloc[5:]
        df[("Nat. / Voile",)] = df[("Nat. / Voile",)].str.replace('\n', '', regex=True)
        df[("Skipper / Bateau",)] = df[("Skipper / Bateau",)].str.replace('\n', ' / ', regex=True)
        #df[("Depuis 30 minutes", "Vitesse")] = df[("Depuis 30 minutes", "Vitesse")].apply(lambda x: remove_unit(x, "kts"))
        date_t = ''.join([f_name.split('_')[1], f_name.split('_')[-1].split('.')[0]])
        df["Date"] = pd.to_datetime(datetime.datetime.strptime(date_t, "%Y%m%d%H%M%S"))
        df_ranks = pd.concat([df_ranks, df])
    except:
        print(f_name)
        break
 
 
df_ranks = df_ranks.set_index(["Date", "Nat. / Voile"]).sort_index(level="Date")
df_ranks.head()


/home/vsoking/anaconda3/envs/geo_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:2901: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(
/home/vsoking/anaconda3/envs/geo_env/lib/python3.8/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)
/home/vsoking/anaconda3/envs/geo_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3172: PerformanceWarning: indexing past lexsort depth may impact performance.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/vsoking/anaconda3/envs/geo_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:2947: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


Rang                        Skipper / Bateau  \
                                                                                
Date                Nat. / Voile                                                
2020-11-08 14:00:00 ESP 33         24      Didac Costa / One Planet One Ocean   
                    FIN 222        31                     Ari Huusela / Stark   
                    FRA 01          7              Jean Le Cam / Yes we Cam !   
                    FRA 09         25  Benjamin Dutreux / OMIA - Water Family   
                    FRA 1000       14           Damien Seguin / Groupe APICIL   

                                         Heure FR    Latitude   Longitude  \
                                                                            
Date                Nat. / Voile                                            
2020-11-08 14:00:00 ESP 33             15:30 FR\n  46°25.59'N  01°48.85'W   
                    FIN 222            15:30 FR\n  46°25.65'N  01°48.21'W   
                    FRA 01             15:30 FR\n  46°24.90'N  01°49.49'W   
                    FRA 09        15:29 FR\n-1min  46°25.47'N  01°48.74'W   
                    FRA 1000      15:28 FR\n-2min  46°24.84'N  01°49.02'W   

                                 Depuis 30 minutes                      \
                                               Cap   Vitesse       VMG   
Date                Nat. / Voile                                         
2020-11-08 14:00:00 ESP 33                    238°  13.4 kts  13.4 kts   
                    FIN 222                   234°  12.1 kts  12.0 kts   
                    FRA 01                    247°  10.7 kts  10.7 kts   
                    FRA 09                    247°  11.6 kts  11.6 kts   
                    FRA 1000                  232°   7.5 kts   7.2 kts   

                                          Depuis le dernier classement  \
                                 Distance                          Cap   
Date                Nat. / Voile                                         
2020-11-08 14:00:00 ESP 33         0.2 nm                         357°   
                    FIN 222        0.2 nm                         358°   
                    FRA 01         0.2 nm                         357°   
                    FRA 09         0.4 nm                         357°   
                    FRA 1000       2.7 nm                         357°   

                                                               \
                                  Vitesse      VMG   Distance   
Date                Nat. / Voile                                
2020-11-08 14:00:00 ESP 33        0.0 kts  0.0 kts  2789.1 nm   
                    FIN 222       0.0 kts  0.0 kts  2789.1 nm   
                    FRA 01        0.0 kts  0.0 kts  2788.4 nm   
                    FRA 09        0.0 kts  0.0 kts  2789.0 nm   
                    FRA 1000      0.0 kts  0.0 kts  2788.4 nm   

                                 Depuis 24 heures                             \
                                              Cap  Vitesse      VMG Distance   
Date                Nat. / Voile                                               
2020-11-08 14:00:00 ESP 33                   193°  0.7 kts  0.6 kts   4.7 nm   
                    FIN 222                  188°  0.2 kts  0.2 kts   4.6 nm   
                    FRA 01                   196°  0.2 kts  0.2 kts   5.5 nm   
                    FRA 09                   192°  0.2 kts  0.2 kts   4.8 nm   
                    FRA 1000                 192°  0.2 kts  0.2 kts   5.4 nm   

                                         DTF     DTL  
                                                      
Date                Nat. / Voile                      
2020-11-08 14:00:00 ESP 33        24295.4 nm  1.5 nm  
                    FIN 222       24295.8 nm  1.9 nm  
                    FRA 01        24294.7 nm  0.8 nm  
                    FRA 09        24295.4 nm  1.5 nm  
                    FRA 1000      

### Liste des abandons

In [6]:
df_ranks.loc[df_ranks["Rang"]=='RET', "Skipper / Bateau"].reset_index(level=1).drop_duplicates(keep="first")

,Nat. / Voile,Skipper / Bateau
Date,,
2020-11-16 17:00:00,FRA 6,Nicolas Troussel / CORUM L'Épargne
2020-12-01 11:00:00,FRA 85,Kevin Escoffier / PRB
2020-12-04 11:00:00,GBR 99,Alex Thomson / HUGO BOSS
2020-12-04 17:00:00,FRA 4,Sébastien Simon / ARKEA PAPREC
2020-12-05 14:00:00,FRA 109,Samantha Davies / Initiatives - Coeur
2020-12-11 17:00:00,FRA 56,Fabrice Amedeo / Newrest - Art et Fenetres
2021-01-10 04:00:00,FRA 27,Isabelle Joschke / MACSF
2021-01-16 21:00:00,FRFRA 69,Sébastien Destremau / Merci


### Nettoyage des données

In [7]:
df_ranks = df_ranks.dropna()
df_ranks['Rang'] = df_ranks['Rang'].astype(int)
df_ranks[("Depuis le dernier classement", "Distance")] = df_ranks[("Depuis le dernier classement", "Distance")].apply(lambda x: remove_unit(x, "nm")).astype(float)
df_ranks[("Depuis 24 heures", "VMG")] = df_ranks[("Depuis 24 heures", "VMG")].apply(lambda x: remove_unit(x, "kts")).astype(float)
df_ranks[("Depuis 24 heures", "Vitesse")] = df_ranks[("Depuis 24 heures", "Vitesse")].apply(lambda x: remove_unit(x, "kts")).astype(float)
df_ranks[("Depuis 24 heures", "Distance")] = df_ranks[("Depuis 24 heures", "Distance")].apply(lambda x: remove_unit(x, "nm")).astype(float)
df_ranks[[("Skipper", ''), ("Bateau", '')]] = df_ranks[("Skipper / Bateau")].str.split('/', expand=True)
df_ranks["Skipper"] = df_ranks["Skipper"].str.lower().str.strip()
df_ranks["Skipper"] = df_ranks["Skipper"].apply(lambda x: " ".join(unicodedata.normalize("NFD", x).encode("ascii", "ignore").decode("utf").split()))
df_spec["Skipper"] = df_spec["Skipper"].str.lower().str.strip()
df_spec["Skipper"] = df_spec["Skipper"].apply(lambda x: " ".join(unicodedata.normalize("NFD", x).encode("ascii", "ignore").decode("utf").split()))#.apply(lambda x: x = "samantha davies" if "davies" in x else x).unique()
df_spec.loc[df_spec["Skipper"].str.contains("davies"), "Skipper"] = "samantha davies"

ind = [i[0]+i[1] for i in df_ranks.columns.tolist()]
df_ranks_flatten = df_ranks.copy()
df_ranks_flatten.columns = pd.Index(ind)

df_merged = df_ranks_flatten.merge(df_spec[["Skipper", "Nombre de dérives"]], left_on="Skipper", right_on="Skipper", how="left")#["Nombre de dérives"]
df_merged = df_merged.set_index(df_ranks.index)

#df_ranks

### Evolution du classement des voiliers durant la course

In [9]:
dates =  pd.date_range(start="2020-11-16 14:00:00", end=df_ranks.index.get_level_values(0)[-1], freq='D')

df = df_ranks[[("Rang",''),("Skipper",'')]].loc[dates].reset_index(level=1)
fig = px.line(df, x=df.index.get_level_values(0), y="Rang",color="Skipper")
fig.update_yaxes(autorange="reversed").show()

### Leaders (pourcentage de temps classé leader)

In [23]:
df = df_ranks.loc[df_ranks["Rang"]==1, "Skipper"].value_counts()
px.pie(values=df, names=df.index.tolist()).show()
print("{} a été leader {} fois \nSuivi de {} qui a été leader {} fois".format(df.index.tolist()[0], df.iloc[0], df.index.tolist()[1], df.iloc[1]))

charlie dalin a été leader 224 fois 
Suivi de yannick bestaven qui a été leader 156 fois


### Distance totale parcourue par Skipper

In [13]:

df_dist_total = df_ranks[[("Depuis le dernier classement", "Distance"),("Skipper",'')]].pivot_table(columns="Skipper", values="Depuis le dernier classement", aggfunc=sum).squeeze().sort_values()
px.bar(df_dist_total, orientation='h', labels={'x':"Distance totale", 'y':"Skipper"}).show()


### Plus grande distance parcourue en 24H

In [15]:
dates = pd.date_range(start=df_ranks.index.get_level_values(0)[0], end=df_ranks.index.get_level_values(0)[-1], freq='D')
df_dist = df_ranks.loc[dates,[("Depuis 24 heures", "Distance"), ("Skipper","")]].droplevel(1).pivot(columns="Skipper", values=("Depuis 24 heures", "Distance")).fillna(0).max().sort_values(ascending=False).squeeze()
px.bar(df_dist).update_layout(yaxis_title="distance (nm) parcourue en 24H").show()

### Meilleure vitesse sur 24H

In [17]:
dates = pd.date_range(start=df_ranks.index.get_level_values(0)[0], end=df_ranks.index.get_level_values(0)[-1], freq='D')
df_dist = df_ranks.loc[dates,[("Depuis 24 heures", "Vitesse"), ("Skipper","")]].droplevel(1).pivot(columns="Skipper", values=("Depuis 24 heures", "Vitesse")).fillna(0).max().sort_values(ascending=False).squeeze()
px.bar(df_dist).update_layout(yaxis_title="Vitesse (kts) en 24H").show()

### Impact du foil sur la vitesse

In [28]:
dates = pd.date_range(start=df_ranks.index.get_level_values(0)[0], end=df_ranks.index.get_level_values(0)[-1], freq='D')
df_dist = df_merged.loc[dates,["Depuis 24 heuresVitesse", "Nombre de dérives"]].reset_index(0)
df_dist["Nombre de dérives"] = df_dist["Nombre de dérives"].str.replace("foiler", "foils")
df_dist["Nombre de dérives"] = df_dist["Nombre de dérives"].str.replace("2 asymétriques", "2")
fig_dist = px.scatter(x=df_dist["Date"], y=df_dist["Depuis 24 heuresVitesse"], color=df_dist["Nombre de dérives"], marginal_y="box" ,labels={'x':"Date", 'y':"Vitesse en 24H"})
fig_dist.show()
print("En observant la distribution, on constae que les bateaux équipés de foils vont légérement plus vite que les autres")

En observant la distribution, on constae que les bateaux équipés de foils vont légérement plus vite que les autres


### Regression linéaire entre classement et la vitesse (VMG)

In [27]:
df = df_ranks[[("Rang",''),("Depuis 24 heures", "VMG")]].unstack().dropna(axis=1).mean().unstack()
fig = px.scatter(y=df.loc["Rang"].squeeze(), x=df.loc[("Depuis 24 heures","VMG")].squeeze(), trendline="ols", labels={'y':"Rang", 'x':"VMG"})
fig.show()


### Route  des voiliers sur la carte

In [60]:
def convert_to_dms(val, orientation):
    d = float(val.split('°')[0]) + float(val.split('°')[1].split("\'")[0])/60
    o = val[-1]
    if orientation == "lat":
        if o == 'S':
            d *= -1
        elif o == 'N':
            pass
        else:
            raise ValueError
    elif orientation == "lon":
        if o == 'W':
            d *= -1
        elif o == 'E':
            pass
        else:
            raise ValueError
    return d

df_nat_skip = df_ranks["Skipper"].reset_index(1).reset_index(drop=True).drop_duplicates(keep="first")
name = lambda x: df_nat_skip.loc[df["Nat. / Voile"] == x,"Skipper"].values[0]


lat = lambda x: df_ranks.loc[(slice(None), x),"Latitude"].apply(lambda x: convert_to_dms(x, "lat"))
lon = lambda y: df_ranks.loc[(slice(None), y),"Longitude"].apply(lambda x: convert_to_dms(x, "lon"))

fig = go.Figure()
fig.update_layout(
    title_text = 'routes',
    showlegend = True,
    geo = go.layout.Geo(
        projection_type = 'orthographic',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
    height=700,
)

fig.add_trace ( go.Scattergeo(text=name("FRA 01"),
        lon = lon("FRA 01"),
        lat = lat("FRA 01"),
        mode = 'lines',
        line = dict(width = 1.5,color = 'green'),
        opacity = 1, name=name("FRA 01")
    ))

fig.add_trace ( go.Scattergeo(text=name("FRA 85"),
        lon = lon("FRA 85"),
        lat = lat("FRA 85"),
        mode = 'lines',
        line = dict(width = 2.5,color = 'red'),
        opacity = 1, name=name("FRA 85")
    ))

fig.add_trace( go.Scattergeo(lon = lon("FRA 17"), lat = lat("FRA 17"), mode = 'lines',
        line = dict(width = 1,color = 'blue'),
        opacity = 1 , name=name("FRA 17")), )

fig.show()



###### Sur la carte ci-dessus, nous pouvons voir la route emprutée par 3 skippers. La route de kevin Escoffier a été subitement interrompue car il a fait naufrage et  heureusement, il a été secouru par Jean Le Cam. 